In [31]:
from flask import Flask, jsonify
import pandas as pd
import json
from collections import defaultdict
from datetime import datetime
import calendar
from aquasentinel import predict_wqi, get_wqc 
import numpy as np
from scipy.stats import zscore
from sklearn.preprocessing import StandardScaler
from flask import request
import logging

In [32]:
data = pd.read_csv("../backend/dataset/indian_water_physicochemical_parameters.csv", encoding='unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
dates = data["date"].values


In [33]:
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2052.0,TUIRIAL UPPER CATCHMENT,MIZORAM,23.6,6.9,7.1,186.0,1.6,0.10,233.0,29.0,2014,2014-12-22
734,2053.0,TUIRIAL LOWER CATCHMENT,MIZORAM,23.5,6.8,7.1,201.0,1.4,0.20,233.0,86.0,2014,2014-12-24
735,1403.0,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",TRIPURA,27.3,6.2,7.8,165.0,2.1,0.00,282.0,340.0,2014,2014-12-25
736,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,29.1,6.4,7.8,158.0,4.1,0.50,458.0,500.0,2014,2014-12-26


In [34]:
import random
from datetime import datetime, timedelta

def generate_dates():
    start_date = datetime(2014, 12, 28)
    end_date = datetime.now()

    while start_date <= end_date:
        yield start_date.strftime("%Y-%m-%d")
        gap = random.randint(1, 7)
        start_date += timedelta(days=gap)

dates = list(generate_dates())

states = data["STATE"].dropna().unique()
locations = data["LOCATIONS"].dropna().unique()
state_station_code = data.groupby('STATE')['STATION CODE'].unique().apply(lambda x: random.choice(x)).to_dict()




max_temp = data["Temp"].max()
min_temp = data["Temp"].min()

max_do = data["D.O. (mg/l)"].max()
min_do = data["D.O. (mg/l)"].min()


max_ph = data["PH"].max()
min_ph = data["PH"].min()


max_cod = data["CONDUCTIVITY (Âµmhos/cm)"].max()
min_cod = data["CONDUCTIVITY (Âµmhos/cm)"].min()


max_bod = data["B.O.D. (mg/l)"].max()
min_bod = data["B.O.D. (mg/l)"].min()

max_nit = data["NITRATENAN N+ NITRITENANN (mg/l)"].max()
min_nit = data["NITRATENAN N+ NITRITENANN (mg/l)"].min()


max_fecal = data["FECAL COLIFORM (MPN/100ml)"].max()
min_fecal = data["FECAL COLIFORM (MPN/100ml)"].min()


max_total_col = data["TOTAL COLIFORM (MPN/100ml)Mean"].max()
min_total_col = data["TOTAL COLIFORM (MPN/100ml)Mean"].min()


for i in dates:
    random_state = random.choice(list(states))
    data.loc[len(data.index)] = [state_station_code[random_state],
                                 random.choice(list(locations)),
                                random_state,
                                round(random.uniform(min_temp, max_temp), 1),
                                round(random.uniform(min_do, max_do), 1),
                                round(random.uniform(min_ph, max_ph), 1),
                                round(random.uniform(min_cod, max_cod), 1),
                                round(random.uniform(min_bod, max_bod), 1),
                                round(random.uniform(min_nit, max_nit), 1),
                                round(random.uniform(min_fecal, max_fecal), 1),
                                round(random.uniform(min_total_col, max_total_col), 1),
                                i[0:4],
                                i]

In [35]:

data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,1399.0,TAMBIRAPARANI AT ARUMUGANERI,GOA,17.8,7.0,6.3,14852.1,44.5,7.80,18027.8,58620.7,2024,2024-03-31
1567,2624.0,"SAL AT ORLIM BRIDGE, ORLIM",HIMACHAL PRADESH,30.4,8.5,6.9,14733.2,22.1,5.10,45358.6,84151.4,2024,2024-04-07
1568,1884.0,"RIVER JAWAI AT JAWAI DAM, SIROHI, RAJASTHAN",HARYANA,25.8,9.3,6.8,10948.9,6.3,2.90,28270.7,31347.8,2024,2024-04-13
1569,2352.0,"CHANDRAGIRI AT PADIYATHADKA, KERALA",ANDHRA PRADESH,24.7,4.3,6.5,13536.6,81.5,3.60,41683.7,83490.4,2024,2024-04-17


In [36]:
data.dtypes

STATION CODE                        float64
LOCATIONS                            object
STATE                                object
Temp                                float64
D.O. (mg/l)                         float64
PH                                  float64
CONDUCTIVITY (Âµmhos/cm)            float64
B.O.D. (mg/l)                       float64
NITRATENAN N+ NITRITENANN (mg/l)    float64
FECAL COLIFORM (MPN/100ml)          float64
TOTAL COLIFORM (MPN/100ml)Mean      float64
year                                 object
date                                 object
dtype: object

In [37]:
data.to_csv("../backend/dataset/data_generation.csv", index=False)

In [38]:
data = pd.read_csv("../backend/dataset/data_generation.csv")

In [39]:
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1566,1399.0,TAMBIRAPARANI AT ARUMUGANERI,GOA,17.8,7.0,6.3,14852.1,44.5,7.80,18027.8,58620.7,2024,2024-03-31
1567,2624.0,"SAL AT ORLIM BRIDGE, ORLIM",HIMACHAL PRADESH,30.4,8.5,6.9,14733.2,22.1,5.10,45358.6,84151.4,2024,2024-04-07
1568,1884.0,"RIVER JAWAI AT JAWAI DAM, SIROHI, RAJASTHAN",HARYANA,25.8,9.3,6.8,10948.9,6.3,2.90,28270.7,31347.8,2024,2024-04-13
1569,2352.0,"CHANDRAGIRI AT PADIYATHADKA, KERALA",ANDHRA PRADESH,24.7,4.3,6.5,13536.6,81.5,3.60,41683.7,83490.4,2024,2024-04-17


In [40]:
data = pd.read_csv("../backend/dataset/indian_water_physicochemical_parameters.csv", encoding='unicode_escape')
data = data.loc[:, ~data.columns.str.contains('^Unnamed')]
dates = data["date"].values

In [41]:
data

,STATION CODE,LOCATIONS,STATE,Temp,D.O. (mg/l),PH,CONDUCTIVITY (Âµmhos/cm),B.O.D. (mg/l),NITRATENAN N+ NITRITENANN (mg/l),FECAL COLIFORM (MPN/100ml),TOTAL COLIFORM (MPN/100ml)Mean,year,date
0,1393.0,DAMANGANGA AT D/S OF MADHUBAN,DAMAN & DIU,31.0,7.1,7.8,151.0,8.0,0.08,233.0,465.0,2012,2012-01-01
1,1150.0,DAMANGANGA AT KACHIGAON U\S AT GIDC WIER,GUJARAT,28.0,7.2,7.4,319.0,0.7,0.18,5.0,10.0,2012,2012-01-04
2,1865.0,RIVER DHADAR AT KOTHADA,GUJARAT,27.0,6.2,8.4,894.0,9.0,0.38,140.0,350.0,2012,2012-01-05
3,1149.0,KOLAK AT PATALIA BDG,GUJARAT,29.0,2.4,7.6,1500.0,6.2,1.31,3.0,13.0,2012,2012-01-06
4,1399.0,ZUARI AT D/S OF PT. WHERE KUMBARJRIA CANAL JOINS,GOA,31.0,5.9,7.2,52.0,2.1,0.08,233.0,90.0,2012,2012-01-07
...,...,...,...,...,...,...,...,...,...,...,...,...,...
733,2052.0,TUIRIAL UPPER CATCHMENT,MIZORAM,23.6,6.9,7.1,186.0,1.6,0.10,233.0,29.0,2014,2014-12-22
734,2053.0,TUIRIAL LOWER CATCHMENT,MIZORAM,23.5,6.8,7.1,201.0,1.4,0.20,233.0,86.0,2014,2014-12-24
735,1403.0,"GUMTI AT U/S SOUTH TRIPURA,TRIPURA",TRIPURA,27.3,6.2,7.8,165.0,2.1,0.00,282.0,340.0,2014,2014-12-25
736,1404.0,"GUMTI AT D/S SOUTH TRIPURA, TRIPURA",TRIPURA,29.1,6.4,7.8,158.0,4.1,0.50,458.0,500.0,2014,2014-12-26


In [42]:
data.fillna(data.median(), inplace=True)
z_scores = data.apply(zscore)
outliers_count = (z_scores.abs() > 3).sum()
df_outliers = np.where(np.abs(z_scores) > 3)[0]
df_filtered = data.drop(data.index[df_outliers])


TypeError: could not convert string to float: 'DAMANGANGA AT D/S OF MADHUBAN'